In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
import numpy as np
from astropy import cosmology as cosmo
from astropy import units as u
from astropy import constants as c
from astropy.visualization import quantity_support
quantity_support()

# Models

In [ ]:
z_models = np.logspace(-3, 1, 101)
abs_Omega_L_models = np.arange(.5, 5.1, .5)

In [ ]:
H0 = 70
rd = 140

In [ ]:
closed_R0_Dh = 1/np.sqrt(1 + abs_Omega_L_models)

'''
# Old fashion (in terms of t/T):

closed_cos_t0_over_T = np.sqrt(1/(1 + abs_Omega_L_models))
closed_sin_t0_over_T = np.sqrt(1 - 1/(1 + abs_Omega_L_models))
closed_t0_over_T = np.arcsin(closed_sin_t0_over_T)

closed_t_over_T = np.arcsin(sin_t0_over_T[:, np.newaxis]/(1+z_models[np.newaxis, :]))
closed_cos_t_over_T = np.cos(closed_t_over_T)
closed_chi = .5*np.log(
    ((1+closed_cos_t_over_T)/(1-closed_cos_t_over_T)
    ) / (
        ((1+closed_cos_t0_over_T)/(1-closed_cos_t0_over_T)))[:, np.newaxis]
)

# In terms of Omega_L and z:
'''

closed_chi = np.arccosh(np.sqrt(1+1/abs_Omega_L_models[:, np.newaxis]) * (1+z_models[np.newaxis, :]))
closed_chi -= np.arccosh(np.sqrt(1+1/abs_Omega_L_models[:, np.newaxis]))
closed_chi /= 1 + abs_Omega_L_models[:, np.newaxis]

closed_Dm_Dh = np.sin(closed_chi)


In [ ]:
LCDM = cosmo.Planck18

LCDM_Dl_Mpc = LCDM.luminosity_distance(z_models).to_value(u.Mpc)
LCDM_mu = 5*np.log10(LCDM_Dl_Mpc/1e-5)

LCDM_Dm_Mpc = LCDM_Dl_Mpc / (1+z_models)
LCDM_rd_Mpc = 147.16

In [ ]:
flat_H0 = 70.2
flat_R0_Mpc = 3e5/flat_H0

flat_Dm_Mpc = flat_R0_Mpc * np.sin(np.log(1+z_models))
flat_rd_Mpc = 139.5

flat_Dl_Mpc = flat_Dm_Mpc*(1+z_models)
flat_mu = 5*np.log10((1+z_models)*flat_Dm_Mpc/1e-5)

In [ ]:
closed_H0 = 70
closed_T_Gyr = 14
closed_t0_over_T = np.arctan((1/(closed_H0*u.km/u.s/u.Mpc)/(closed_T_Gyr*u.Gyr)).to_value(''))
closed_t0_Gyr = closed_T_Gyr * closed_t0_over_T

sin_t0_over_T = np.sin(closed_t0_over_T)
cos_t0_over_T = np.cos(closed_t0_over_T)
closed_R0_Mpc =  c.c.to_value(u.Mpc/u.Gyr) * closed_T_Gyr * sin_t0_over_T

closed_t_over_T = np.arcsin(sin_t0_over_T/(1+z_models))
cos_t_over_T = np.cos(closed_t_over_T)
closed_chi = .5*np.log(
    ((1+cos_t_over_T)/(1-cos_t_over_T)) / ((1+cos_t0_over_T)/(1-cos_t0_over_T))
)


closed_Dm_Mpc = closed_R0_Mpc * np.sin(closed_chi)
closed_rd_Mpc = 139.5

closed_Dl_Mpc = closed_Dm_Mpc*(1+z_models)
closed_mu = 5*np.log10((1+z_models)*closed_Dm_Mpc/1e-5)

In [ ]:
open_H0 = 73
open_T_Gyr = 20
open_t0_over_T = np.arctanh((1/(open_H0*u.km/u.s/u.Mpc)/(open_T_Gyr*u.Gyr)).to_value(''))
open_t0_Gyr = open_T_Gyr * open_t0_over_T

sinh_t0_over_T = np.sinh(open_t0_over_T)
cosh_t0_over_T = np.cosh(open_t0_over_T)
open_R0_Mpc =  c.c.to_value(u.Mpc/u.Gyr) * open_T_Gyr * sinh_t0_over_T

open_t_over_T = np.arcsinh(sinh_t0_over_T/(1+z_models))
cosh_t_over_T = np.cosh(open_t_over_T)
open_chi = .5*np.log(
    ((1+cosh_t_over_T)/(1-cosh_t_over_T)) / ((1+cosh_t0_over_T)/(1-cosh_t0_over_T))
)


open_Dm_Mpc = open_R0_Mpc * np.sin(open_chi)
open_rd_Mpc = 139.5

open_Dl_Mpc = open_Dm_Mpc*(1+z_models)
open_mu = 5*np.log10((1+z_models)*open_Dm_Mpc/1e-5)

# Data

In [ ]:
SN_z, SN_err_z, SN_mu, SN_err_mu = np.loadtxt('data/Pantheon+_Data.dat', usecols=(2, 3, 10, 11), unpack=True, skiprows=1)
GRB1 = np.loadtxt('data/GRB_RNN.dat', skiprows=1)
GRB2 = np.loadtxt('data/GRB_RNN_2.dat', skiprows=1)
GRB_z = np.concatenate([GRB1[:, 0], GRB1[:, 3], GRB1[:, 6], GRB2[:, 0]])
GRB_mu = np.concatenate([GRB1[:, 1], GRB1[:, 4], GRB1[:, 7], GRB2[:, 1]])
GRB_err_mu = np.concatenate([GRB1[:, 2], GRB1[:, 5], GRB1[:, 8], GRB2[:, 2]])
QSO_z, QSO_mu, QSO_err_mu = np.loadtxt('data/table3QSO.dat', usecols=(3, 11, 12), unpack=True, skiprows=1)


def Dl_Mpc(mu):
    return 1e-5 * 10**(.2*mu)

SN_Dl = Dl_Mpc(SN_mu)
GRB_Dl = Dl_Mpc(GRB_mu)
QSO_Dl = Dl_Mpc(QSO_mu)


all_z = np.concatenate([SN_z, QSO_z, GRB_z])
all_mu = np.concatenate([SN_mu, QSO_mu, GRB_mu])
all_err_mu = np.concatenate([SN_err_mu, QSO_err_mu, GRB_err_mu])

#all_z = SN_z
#all_mu = SN_mu
#all_err_mu = SN_err_mu
#all_z = QSO_z
#all_mu = QSO_mu
#all_err_mu = QSO_err_mu
#all_z = GRB_z
#all_mu = GRB_mu
#all_err_mu = GRB_err_mu


In [ ]:
thetaBAO = np.array([
    0.11,19.8,3.26,
    0.235,9.06,0.23,
    0.365,6.33,0.22,
    0.45,4.77,0.17,
    0.47,5.02,0.25,
    0.49,4.99,0.21,
    0.51,4.81,0.17,
    0.53,4.29,0.30,
    0.55,4.25,0.25,
    0.57,4.59,0.36,
    0.59,4.39,0.33,
    0.61,3.85,0.31,
    0.63,3.90,0.43,
    0.65,3.55,0.16,
    2.225,1.77,0.31])

theta_z = thetaBAO[::3]
theta_err_z = np.array([0.005,0.035,0.025,0.01,0.005,0.01,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.01,0.025])
theta_angle_deg = thetaBAO[1::3]
theta_relative_err = thetaBAO[2::3] / theta_angle_deg
theta_DM = 180/np.pi/theta_angle_deg

In [ ]:
plt.close('observables')
fig = plt.figure('observables', figsize=(7.5, 7.5))
axes = fig.subplots(nrows=2, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

ax = axes[0, 0]
#ax.set_ylabel(r'$\mu$')
ax.set_ylabel(r'$D_L / z$ [Mpc]')
ax.set_yscale('log')
ax.set_ylim(500, 2e4)

'''
ax.errorbar(QSO_z, QSO_mu, yerr=QSO_err_mu, fmt='r.', alpha=.05)
ax.errorbar(GRB_z, GRB_mu, yerr=GRB_err_mu, fmt='g.', alpha=.1)
ax.errorbar(SN_z, SN_mu, xerr=SN_err_z, yerr=SN_err_mu, fmt='c.', alpha=.05)

ax.plot(z_models, LCDM_mu, 'k--')
ax.plot(z_models, flat_mu, 'k-')
'''

y = QSO_Dl/QSO_z
ax.errorbar(QSO_z, y, yerr=QSO_err_mu*y/2, fmt='y.', alpha=.2)
y = GRB_Dl/GRB_z
ax.errorbar(GRB_z, y, yerr=GRB_err_mu*y/2, fmt='g.', alpha=.1)
y = SN_Dl/SN_z
ax.errorbar(SN_z, y, xerr=SN_err_z, yerr=SN_err_mu*y/2, fmt='c.', alpha=.05)

ax.plot(z_models, LCDM_Dl_Mpc/z_models, 'k--')
ax.plot(z_models, flat_Dl_Mpc/z_models, 'k-')
ax.plot(z_models, closed_Dl_Mpc/z_models, 'k:')
ax.plot(z_models, open_Dl_Mpc/z_models, 'k-.')

for i in range(abs_Omega_L_models.size):
    ax.plot(z_models, 3e5/H0 * closed_Dm_Dh[i]*(1+z_models)/z_models, 'k:', label=f'$\Omega_\Lambda$ = -{abs_Omega_L_models[i]}')
ax.legend()

ax = axes[1, 0]
ax.set_ylabel(r'$D_M / z r_d$')
ax.set_yscale('log')
ax.set_ylim(.5, 50)

y = theta_DM/theta_z
ax.errorbar(theta_z, y, xerr=theta_err_z, yerr=theta_relative_err*y, fmt='c.', alpha=.5)

ax.plot(z_models, LCDM_Dm_Mpc/LCDM_rd_Mpc/z_models, 'k--')
ax.plot(z_models, flat_Dm_Mpc/flat_rd_Mpc/z_models, 'k-')
ax.plot(z_models, closed_Dm_Mpc/closed_rd_Mpc/z_models, 'k:')
ax.plot(z_models, open_Dm_Mpc/open_rd_Mpc/z_models, 'k-.')


ax = axes[-1, 0]
ax.set_xlabel(r'redshift')
ax.set_xscale('log')
ax.set_xlim(.001, 10)

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()

# Fit

In [ ]:
def chi2(model, data_z, data, errors):
    return np.mean(((data-np.interp(data_z, z_models, model))/errors)**2)

def chi2_mu(model):
    return chi2(
        model,
        np.concatenate([SN_z, GRB_z, QSO_z]),
        np.concatenate([SN_mu, GRB_mu, QSO_mu]),
        np.concatenate([SN_err_mu, GRB_err_mu, QSO_err_mu]),
               )

print('        LCDM   open   flat  closed')
print('Total: {:.3f} {:.3f} {:.3f} {:.3f}'.format( chi2_mu(LCDM_mu), chi2_mu(open_mu), chi2_mu(flat_mu), chi2_mu(closed_mu)))
print('SN:    {:.4f} {:.4f} {:.4f} {:.4f}'.format(
      chi2(LCDM_mu, SN_z, SN_mu, SN_err_mu),
      chi2(open_mu, SN_z, SN_mu, SN_err_mu),
      chi2(flat_mu, SN_z, SN_mu, SN_err_mu),
      chi2(closed_mu, SN_z, SN_mu, SN_err_mu),
     ))
print('GRB:   {:.4f} {:.4f} {:.4f} {:.4f}'.format(
      chi2(LCDM_mu, GRB_z, GRB_mu, GRB_err_mu),
      chi2(open_mu, GRB_z, GRB_mu, GRB_err_mu),
      chi2(flat_mu, GRB_z, GRB_mu, GRB_err_mu),
      chi2(closed_mu, GRB_z, GRB_mu, GRB_err_mu),
     ))
print('QSO:   {:.3f} {:.3f} {:.3f} {:.3f}'.format(
      chi2(LCDM_mu, QSO_z, QSO_mu, QSO_err_mu),
      chi2(open_mu, QSO_z, QSO_mu, QSO_err_mu),
      chi2(flat_mu, QSO_z, QSO_mu, QSO_err_mu),
      chi2(closed_mu, QSO_z, QSO_mu, QSO_err_mu),
     ))